# SpatialDE: automatic expression histology (AEH) report

In [ ]:
import os
print("Files in use:")
print(f"FILE='{os.path.abspath(FILE)}'")

In [ ]:
# Import packages
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import pandas as pd
from matplotlib.pyplot import rc_context

#### Plotting settings

In [ ]:
# plot settings
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)
hist_dims = (10,3)

### Read data

In [ ]:
adata = sc.read_h5ad(filename=FILE)

---

## Spatial patterns

#### Distribution of length-scale (l) for significant genes

In [ ]:
signif_res = adata.uns['spatialDE'][adata.uns['spatialDE'].is_signif]
df_signif_res = signif_res['l'].value_counts().to_frame(name="counts")
df_signif_res.reset_index(level=0, inplace=True)
df_signif_res.columns = ['length-scale (l)', 'counts']
df_signif_res

In [ ]:
df_signif_res.sort_values('length-scale (l)', inplace=True)
df_signif_res['length-scale (l)'] = [ np.round(l, 3) for l in df_signif_res['length-scale (l)']]
plt = df_signif_res.plot.bar(x='length-scale (l)', y='counts', rot=45)

In [ ]:
print("Median: {}".format(np.median(signif_res['l'])))
print("Mean:\t{} +/- {}".format(np.mean(signif_res['l']), np.std(signif_res['l'])))

#### Distribution of spatial patterns
Adjusting the number of patterns (c) and changing the length-scale (l) may improve detecting spatial patterns

In [ ]:
# add patterns to obs
for pattern in adata.uns['spatialDE_AEH_patterns'].keys():
    patname = 'n_spatialDE_AEH_pattern' + pattern
    adata.obs[patname] = adata.uns['spatialDE_AEH_patterns'][pattern]
    
patterns = ['n_spatialDE_AEH_pattern' + pattern for pattern in adata.uns['spatialDE_AEH_patterns'].keys()]

if 'spatial' in adata.uns.keys():
    sc.pl.spatial(adata, img_key="hires", color=patterns, alpha=1, color_map='OrRd')
else:
    fig_width= 10
    sp_width = np.max(adata.obsm['X_spatial'][:,0]) - np.min(adata.obsm['X_spatial'][:,0])
    sp_height = np.max(adata.obsm['X_spatial'][:,1]) - np.min(adata.obsm['X_spatial'][:,1])
    ratio_hw = sp_height / sp_width
    fig_height = int(ratio_hw * fig_width)
    with rc_context({'figure.figsize': (fig_width, fig_height)}):
        sc.pl.embedding(adata, basis="X_spatial", color=patterns, alpha=1, color_map='OrRd')

In [ ]:
print("Using length-scale l={} and c={} patterns".format(adata.uns['spatialDE_parameters']['l'], adata.uns['spatialDE_parameters']['c']))

#### Assignment of genes to spatial patterns

In [ ]:
for i in adata.uns['spatialDE_AEH_histology_results'].sort_values('pattern').pattern.unique():
    print('Pattern {}'.format(i))
    print(', '.join(adata.uns['spatialDE_AEH_histology_results'].query('pattern == @i').sort_values('membership')['g'].tolist()))
    print()